In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from modules.Utils.utils import DummyLogger
import sqlite3
from tqdm import tqdm
import time
import subprocess
import matplotlib.dates as mdates
from datetime import datetime
from astropy.table import Table
from tqdm.notebook import tqdm_notebook
%matplotlib inline

In [8]:
def get_keyword_types(level):
        """
        Returns a dictionary of the data types for keywords at the L0/2D/L1/L2 or 
        L0_telemetry level.
        """
        
        # L0 PRIMARY header    
        if level == 'L0':
            keywords_csv='/code/KPF-Pipeline/static/tsdb_keywords/l0_primary_keywords.csv'
            df_keywords = pd.read_csv(keywords_csv, delimiter='|', dtype=str)
            keyword_types = dict(zip(df_keywords['keyword'], df_keywords['datatype']))
             
        # 2D PRIMARY header    
        elif level == '2D':
            keywords_csv='/code/KPF-Pipeline/static/tsdb_keywords/d2_primary_keywords.csv'
            df_keywords = pd.read_csv(keywords_csv, delimiter='|', dtype=str)
            keyword_types = dict(zip(df_keywords['keyword'], df_keywords['datatype']))

        # L1 PRIMARY header    
        elif level == 'L1':
            keywords_csv='/code/KPF-Pipeline/static/tsdb_keywords/l1_primary_keywords.csv'
            df_keywords = pd.read_csv(keywords_csv, delimiter='|', dtype=str)
            keyword_types = dict(zip(df_keywords['keyword'], df_keywords['datatype']))
        
        # L2 PRIMARY header    
        elif level == 'L2':
            keywords_csv='/code/KPF-Pipeline/static/tsdb_keywords/l2_primary_keywords.csv'
            df_keywords = pd.read_csv(keywords_csv, delimiter='|', dtype=str)
            keyword_types = dict(zip(df_keywords['keyword'], df_keywords['datatype']))

        # L0 TELEMETRY extension
        elif level == 'L0_telemetry':
            keywords_csv='/code/KPF-Pipeline/static/tsdb_keywords/l0_telemetry_keywords.csv'
            df_keywords = pd.read_csv(keywords_csv, delimiter='|', dtype=str)
            keyword_types = dict(zip(df_keywords['keyword'], df_keywords['datatype']))

        # L2 RV extension    
        elif level == 'L2_RV_header':
            keywords_csv='/code/KPF-Pipeline/static/tsdb_keywords/l2_rv_keywords.csv'
            df_keywords = pd.read_csv(keywords_csv, delimiter='|', dtype=str)
            keyword_types = dict(zip(df_keywords['keyword'], df_keywords['datatype']))

        # L2 CCF extension    
        elif level == 'L2_CCF_header':
            keywords_csv='/code/KPF-Pipeline/static/tsdb_keywords/l2_green_ccf_keywords.csv'
            df_keywords = pd.read_csv(keywords_csv, delimiter='|', dtype=str)
            keyword_types = dict(zip(df_keywords['keyword'], df_keywords['datatype']))

        # L2 RV data    
        elif level == 'L2_RV':
            prefixes = ['RV1', 'RV2', 'RV3', 'RVS', 'ERVS', 'RVC', 'ERVC', 'RVY', 'ERVY', 'CCFBJD', 'BCRV', 'CCFW']
            nums = [f"{i:02d}" for i in range(67)]
            keyword_types = {f"{prefix}{num}": 'REAL' for num in nums for prefix in prefixes}

        else:
            keyword_types = {}

        return keyword_types

def map_data_type_to_sql(dtype):
        """
        Function to map the data types specified in get_keyword_types to sqlite3
        data types.
        """
        return {
            'int': 'INTEGER',
            'float': 'REAL',
            'bool': 'BOOLEAN',
            'datetime': 'TEXT',  # SQLite does not have a native datetime type
            'string': 'TEXT'
        }.get(dtype, 'TEXT')

In [6]:
logger = DummyLogger()
logger.info('Starting AnalyzeTimeSeries')
tqdm = tqdm_notebook
logger.info('Jupyter Notebook environment detected.')
db_path = 'kpf_ts_oct2024_v2.db'
logger.info('Path of database file: ' + os.path.abspath(db_path))
base_dir='/data/L0'
logger.info('Base data directory: ' + base_dir)
L0_header_keyword_types     = get_keyword_types(level='L0')
L0_telemetry_types          = get_keyword_types(level='L0_telemetry')
D2_header_keyword_types     = get_keyword_types(level='2D')
L1_header_keyword_types     = get_keyword_types(level='L1')
L2_header_keyword_types     = get_keyword_types(level='L2')
L2_CCF_header_keyword_types = get_keyword_types(level='L2_CCF_header')
L2_RV_header_keyword_types  = get_keyword_types(level='L2_RV_header')
L2_RV_keyword_types         = get_keyword_types(level='L2_RV')

INFO: Starting AnalyzeTimeSeries
INFO: Jupyter Notebook environment detected.
INFO: Path of database file: /code/KPF-Pipeline/kpf_ts_oct2024_v2.db
INFO: Base data directory: /data/L0


In [9]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
conn.execute("PRAGMA journal_mode=WAL")
conn.execute("PRAGMA wal_autocheckpoint")
cursor.execute("PRAGMA cache_size = -2000000;")

# Define columns for each file type
L0_header_cols     = [f'"{key}" {map_data_type_to_sql(dtype)}' for key, dtype in L0_header_keyword_types.items()]
L0_telemetry_cols  = [f'"{key}" {map_data_type_to_sql(dtype)}' for key, dtype in L0_telemetry_types.items()]
D2_header_cols     = [f'"{key}" {map_data_type_to_sql(dtype)}' for key, dtype in D2_header_keyword_types.items()]
L1_header_cols     = [f'"{key}" {map_data_type_to_sql(dtype)}' for key, dtype in L1_header_keyword_types.items()]
L2_header_cols     = [f'"{key}" {map_data_type_to_sql(dtype)}' for key, dtype in L2_header_keyword_types.items()]
L2_CCF_header_cols = [f'"{key}" {map_data_type_to_sql(dtype)}' for key, dtype in L2_CCF_header_keyword_types.items()]
L2_RV_header_cols  = [f'"{key}" {map_data_type_to_sql(dtype)}' for key, dtype in L2_RV_header_keyword_types.items()]
L2_RV_cols         = [f'"{key}" {map_data_type_to_sql(dtype)}' for key, dtype in L2_RV_keyword_types.items()]
cols = L0_header_cols + L0_telemetry_cols + D2_header_cols + L1_header_cols + L2_header_cols + L2_CCF_header_cols + L2_RV_header_cols + L2_RV_cols
cols += ['"datecode" TEXT', '"ObsID" TEXT']
cols += ['"L0_filename" TEXT', '"D2_filename" TEXT', '"L1_filename" TEXT', '"L2_filename" TEXT', ]
cols += ['"L0_header_read_time" TEXT', '"D2_header_read_time" TEXT', '"L1_header_read_time" TEXT', '"L2_header_read_time" TEXT', ]
cols += ['"Source" TEXT']
create_table_query = f'CREATE TABLE IF NOT EXISTS kpfdb ({", ".join(cols)}, UNIQUE(ObsID))'
cursor.execute(create_table_query)

# Define indexed columns
index_commands = [
    ('CREATE UNIQUE INDEX idx_ObsID       ON kpfdb ("ObsID");',       'idx_ObsID'),
    ('CREATE UNIQUE INDEX idx_L0_filename ON kpfdb ("L0_filename");', 'idx_L0_filename'),
    ('CREATE UNIQUE INDEX idx_D2_filename ON kpfdb ("D2_filename");', 'idx_D2_filename'),
    ('CREATE UNIQUE INDEX idx_L1_filename ON kpfdb ("L1_filename");', 'idx_L1_filename'),
    ('CREATE UNIQUE INDEX idx_L2_filename ON kpfdb ("L2_filename");', 'idx_L2_filename'),
    ('CREATE INDEX idx_FIUMODE ON kpfdb ("FIUMODE");', 'idx_FIUMODE'),
    ('CREATE INDEX idx_OBJECT ON kpfdb ("OBJECT");', 'idx_OBJECT'),
    ('CREATE INDEX idx_DATE_MID ON kpfdb ("DATE-MID");', 'idx_DATE_MID'),
]

# Iterate and create indexes if they don't exist
for command, index_name in index_commands:
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='index' AND name='{index_name}';")
    if cursor.fetchone() is None:
        cursor.execute(command)

conn.commit()
conn.close()
logger.info("Primary table 'kpfdb' created/updated successfully.")

INFO: Primary table 'kpfdb' created/updated successfully.


In [11]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

create_meta_table_query = """
    CREATE TABLE IF NOT EXISTS kpfdb_metadata (
        keyword     TEXT NOT NULL PRIMARY KEY,
        datatype   TEXT,
        description TEXT,
        units       TEXT,
        source      TEXT
    )
"""
cursor.execute(create_meta_table_query)
conn.commit()

def load_keyword_csv(csv_path, source_label):
    """
    Helper function to read a CSV file and return a DataFrame with columns:
       keyword | datatype | unit | description | source
    """
    df = pd.read_csv(csv_path, delimiter='|', dtype=str)
    df['source'] = source_label
    df = df[['keyword', 'datatype', 'unit', 'description', 'source']]
    return df

# Load keywords from CSV files for multiple levels
df_der   = load_keyword_csv('/code/KPF-Pipeline/static/tsdb_keywords/derived_keywords.csv',      source_label='Derived Keywords')
df_l0    = load_keyword_csv('/code/KPF-Pipeline/static/tsdb_keywords/l0_primary_keywords.csv',   source_label='L0 PRIMARY Header')
df_2d    = load_keyword_csv('/code/KPF-Pipeline/static/tsdb_keywords/d2_primary_keywords.csv',   source_label='2D PRIMARY Header')
df_l1    = load_keyword_csv('/code/KPF-Pipeline/static/tsdb_keywords/l1_primary_keywords.csv',   source_label='L1 PRIMARY Header')
df_l2    = load_keyword_csv('/code/KPF-Pipeline/static/tsdb_keywords/l2_primary_keywords.csv',   source_label='L2 PRIMARY Header')
df_l0t   = load_keyword_csv('/code/KPF-Pipeline/static/tsdb_keywords/l0_telemetry_keywords.csv', source_label='L0 TELEMETRY Extension')
df_l2rv  = load_keyword_csv('/code/KPF-Pipeline/static/tsdb_keywords/l2_rv_keywords.csv',        source_label='L2 RV Header')
df_l2ccf = load_keyword_csv('/code/KPF-Pipeline/static/tsdb_keywords/l2_green_ccf_keywords.csv', source_label='L2 CCF Header')

# Build the RV prefix keywords (the 8th source)
prefixes = ['RV1','RV2','RV3','RVS','ERVS','RVC','ERVC','RVY','ERVY','CCFBJD','BCRV','CCFW']
units    = ['km/s','km/s','km/s','km/s','km/s','km/s','km/s','km/s','km/s','days','km/s','None']
descs    = ['RV for SCI1 order ', 'RV for SCI2 order ', 'RV for SCI3 order ','RV for SCI order ', 'Error in RV for SCI order ', 'RV for CAL order ','Error in RV for CAL order ', 'RV for SKY order ',  'Error in RV for SKY order ','BJD for order ','Barycentric RV for order ','CCF weight for order ']
nums = [f"{i:02d}" for i in range(67)]

prefix_unit_map = dict(zip(prefixes, units))
prefix_desc_map = dict(zip(prefixes, descs))

rv_entries = []
for prefix in prefixes:
    for num in nums:
        kw   = f"{prefix}{num}"
        dtyp = "REAL"
        desc = f"{prefix_desc_map[prefix]}{num}"
        unt  = prefix_unit_map[prefix]
        rv_entries.append([kw, dtyp, unt, desc, 'L2 RV Extension'])

df_rv = pd.DataFrame(rv_entries, columns=['keyword','datatype','unit','description','source'])

# Combine all dataframes and remove duplicate keywords
df_all = pd.concat([
    df_der, df_l0, df_2d, df_l1, df_l2, df_l0t, df_l2rv, df_l2ccf, df_rv
], ignore_index=True)
df_all.drop_duplicates(subset='keyword', keep='first', inplace=True)

# Insert (or replace) into kpfdb_metadata
insert_query = """
    INSERT OR REPLACE INTO kpfdb_metadata
    (keyword, datatype, description, units, source)
    VALUES (?, ?, ?, ?, ?)
"""

for _, row in df_all.iterrows():
    cursor.execute(
        insert_query,
        (row['keyword'], row['datatype'], row['description'], row['unit'], row['source'])
    )

conn.commit()
conn.close()

logger.info("Metadata table 'kpfdb_metadata' created/updated successfully.")

INFO: Metadata table 'kpfdb_metadata' created/updated successfully.


In [12]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute('SELECT COUNT(*) FROM kpfdb')
nrows = cursor.fetchone()[0]
cursor.execute('PRAGMA table_info(kpfdb)')
ncolumns = len(cursor.fetchall())
cursor.execute('SELECT MAX(MAX(L0_header_read_time),MAX(L1_header_read_time)) FROM kpfdb')
most_recent_read_time = cursor.fetchone()[0]
cursor.execute('SELECT MIN(datecode) FROM kpfdb')
earliest_datecode = cursor.fetchone()[0]
cursor.execute('SELECT MAX(datecode) FROM kpfdb')
latest_datecode = cursor.fetchone()[0]
cursor.execute('SELECT COUNT(DISTINCT datecode) FROM kpfdb')
unique_datecodes_count = cursor.fetchone()[0]
conn.close()
logger.info(f"Summary: {nrows} obs x {ncolumns} cols over {unique_datecodes_count} days in {earliest_datecode}-{latest_datecode}; updated {most_recent_read_time}")

INFO: Summary: 14928 obs x 1188 cols over 27 days in 20241001-20241027; updated 2025-02-06 18:56:34


In [13]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
custom_order = [
    "Derived Keywords",
    "L0 PRIMARY Header",
    "2D PRIMARY Header",
    "L1 PRIMARY Header",
    "L2 PRIMARY Header",
    "L0 TELEMETRY Extension",
    "L2 RV Header",
    "L2 RV Extension"
]

# Define fixed column widths (adjust as needed)
col_width_keyword  = 35
col_width_datatype = 9
col_width_units    = 9
col_width_desc     = 90

# For each source in the custom order, fetch and print its rows
for src in custom_order:
    # Query rows for this source
    cursor.execute(
        """
        SELECT keyword, datatype, units, description
        FROM kpfdb_metadata
        WHERE source = ?
        ORDER BY keyword;
        """,
        (src,)
    )
    rows = cursor.fetchall()

    # Skip if there are no rows for this source
    if not rows:
        continue

    # Print section header
    print(f"{src}:")
    print("-" * 90)

    # Print table header
    print(
        f"{'Keyword':<{col_width_keyword}} "
        f"{'Datatype':<{col_width_datatype}} "
        f"{'Units':<{col_width_units}} "
        f"{'Description':<{col_width_desc}}"
    )
    print("-" * 90)

    # Print each row in a fixed-width format
    for keyword, datatype, units, description in rows:
        # Convert None to an empty string to avoid formatting errors
        keyword_str   = keyword if keyword else ""
        datatype_str  = datatype if datatype else ""
        units_str     = units if units else ""
        desc_str      = description if description else ""

        print(
            f"{keyword_str:<{col_width_keyword}} "
            f"{datatype_str:<{col_width_datatype}} "
            f"{units_str:<{col_width_units}} "
            f"{desc_str:<{col_width_desc}}"
        )
    print()  # Blank line after each group

conn.close()

Derived Keywords:
------------------------------------------------------------------------------------------
Keyword                             Datatype  Units     Description                                                                               
------------------------------------------------------------------------------------------
D2_filename                         string    None      Full path to 2D file                                                                      
D2_header_read_time                 string    None      Last modification of 2D file                                                              
L0_filename                         string    None      Full path to L0 file                                                                      
L0_header_read_time                 string    None      Last modification of L0 file                                                              
L1_filename                         string    None      Full path

In [27]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(
    """
    SELECT keyword, description
    FROM kpfdb_metadata
    WHERE source = "L2 RV Extension"
    ORDER BY keyword;
    """)
rows = cursor.fetchall()
keywords = [f'"{keyword}"' for keyword, description in rows if keyword]
query = f"""SELECT ObsID, OBJECT, [DATE-MID], 
    {', '.join(keywords)} 
    FROM kpfdb
    WHERE OBJECT LIKE '%autocal-etalon%' OR OBJECT LIKE '%autocal-lfc%'"""
df = pd.read_sql_query(query, conn)
conn.close()
df

,ObsID,OBJECT,DATE-MID,BCRV00,BCRV01,BCRV02,BCRV03,BCRV04,BCRV05,BCRV06,...,RVY57,RVY58,RVY59,RVY60,RVY61,RVY62,RVY63,RVY64,RVY65,RVY66
0,KP.20241008.31459.57,autocal-etalon-all-night,2024-10-08T08:44:22.797,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,KP.20241008.25014.55,autocal-etalon-all-night,2024-10-08T06:57:24.557,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,KP.20241001.10301.86,autocal-etalon-all-eve,2024-10-01T02:52:11.874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.00130827472970206,-0.00760824229698606,-0.00199172234737003,-0.00288828775534678,0.00130673530612248,0.00239084636752753,-0.00277932967909856,0.00388399486215854,0.00107319289230412,0.00489830505736588
3,KP.20241001.05482.41,autocal-etalon-all-eve,2024-10-01T01:31:52.415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.00224184125509182,-0.00428694935758994,-0.00483801301505781,-0.000246882693443585,0.00260212945562324,0.00258675911999257,0.00117222706711054,0.00387650323305498,0.0034103417167276,0.000925316651239199
4,KP.20241001.05590.94,autocal-etalon-all-eve,2024-10-01T01:33:40.948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.00190020597200976,-0.000468424352054419,0.00146811521912597,-0.0026004788985428,0.00157809666175687,0.00291910107457394,-0.00250909687877149,-0.000867318913418442,0.00220333217344753,0.000436107171850197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,KP.20241027.63182.84,autocal-etalon-all-morn,2024-10-27T17:33:32.853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.00791522625983338,-0.00586905784003962,-0.00574052821185081,-0.00751216982437125,-0.00285689644627849,0.00245974915344221,-0.00474996546347014,0.00614998362912106,0.00755059065075699,0.00589962943491402
3164,KP.20241027.63291.25,autocal-etalon-all-morn,2024-10-27T17:35:21.291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.00864527133793613,-0.0068138206668233,-0.00725368681482034,-0.0111697883973747,-0.00475306747322285,3.34086342711982e-05,-0.00365492795937209,-0.00513820144347276,-0.00206757142409738,0.0038239161661865
3165,KP.20241027.66670.90,autocal-etalon-all-morn,2024-10-27T18:31:40.930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.00603537383312715,-0.00319593219181413,-0.00350037425037564,-0.0120332986797215,-0.00829905771269683,-0.00204213839521049,-0.00623018272785283,-0.000996285690529471,0.00829793752606305,0.00451522317172048
3166,KP.20241027.66779.59,autocal-etalon-all-morn,2024-10-27T18:33:29.617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.00781114511676231,-0.00448534350746926,-0.00339354573295336,-0.0080063357693861,-0.00601095916237446,-0.0041079507942149,-0.00375588151835398,0.00281634082943879,0.0102817829560516,0.00119791228699179


In [30]:
#identifying columns to reshape (the ones ending with digits)
columns_to_reshape = [col for col in df.columns if col[-2:].isdigit()]

#extracting ORDER and prefix from the column names
order_list = []
prefix_list = []
for col in columns_to_reshape:
    prefix = col[:-2]  # Extract the prefix (e.g., 'BCRV')
    order = col[-2:]   # Extract the order (e.g., '00')
    prefix_list.append(prefix)
    order_list.append(order)

#creating new dataframe with ORDER 
unique_orders = sorted(set(order_list))  # Unique ORDER values
new_df = pd.DataFrame({
    'ORDER': unique_orders,
})

#add preserved columns to the new dataframe
preserved_columns = ['ObsID', 'OBJECT', 'DATE-MID']
temp_df = df[preserved_columns].assign(key=1)
new_df = new_df.assign(key=1).merge(temp_df, on='key', how='outer').drop('key', axis=1)

#adding columns for each unique prefix
unique_prefixes = set(prefix_list)
for prefix in unique_prefixes:
    new_df[prefix] = None  # Initialize the column with None

#populating the new DataFrame with values from the original DataFrame
for i, col in enumerate(columns_to_reshape):
    prefix = prefix_list[i]
    order = order_list[i]
    new_df.loc[new_df['ORDER'] == order, prefix] = df[col].values

#sorting the DataFrame by preserved columns and ORDER
new_df = new_df.sort_values(by=preserved_columns + ['ORDER']).reset_index(drop=True)

new_df

,ORDER,ObsID,OBJECT,DATE-MID,CCFW,RV1,RVC,RV3,ERVC,RV2,RVY,CCFBJD,ERVS,RVS,ERVY,BCRV
0,00,KP.20241001.05482.41,autocal-etalon-all-eve,2024-10-01T01:31:52.415,0.0,0.0,0.0,0.0,0.00307450614085213,0.0,0.0,2460584.57029073,0.00129724269887308,0.0,0.00327160792336159,0.0
1,01,KP.20241001.05482.41,autocal-etalon-all-eve,2024-10-01T01:31:52.415,0.0,0.0,0.0,0.0,0.00341173275364434,0.0,0.0,2460584.57029073,0.00136478474714885,0.0,0.00274739160176658,0.0
2,02,KP.20241001.05482.41,autocal-etalon-all-eve,2024-10-01T01:31:52.415,0.0,0.0,0.0,0.0,0.0028710222959318,0.0,0.0,2460584.57029073,0.00115560348324038,0.0,0.00242423577718878,0.0
3,03,KP.20241001.05482.41,autocal-etalon-all-eve,2024-10-01T01:31:52.415,0.0,0.0,0.0,0.0,0.00220358314647423,0.0,0.0,2460584.57029073,0.000848307937247495,0.0,0.00172502738902567,0.0
4,04,KP.20241001.05482.41,autocal-etalon-all-eve,2024-10-01T01:31:52.415,0.0,0.0,0.0,0.0,0.0020919641511566,0.0,0.0,2460584.57029073,0.000794650404753924,0.0,0.00164845659546253,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212251,62,KP.20241027.66888.25,autocal-etalon-all-morn,2024-10-27T18:35:18.259,1.0,0.0038917280487095,-0.0135114168186626,-0.0120086481831451,0.00128127171975815,-0.00570131900780981,-0.00437933189624569,2460611.28001172,0.000406201046472387,-0.00481618764845624,0.00109891690663521,0.0
212252,63,KP.20241027.66888.25,autocal-etalon-all-morn,2024-10-27T18:35:18.259,1.0,0.0015744211310989,0.00386397714575651,-0.0178982567718566,0.00119140824299686,-0.00502737218686882,-0.0031927158966143,2460611.28001172,0.000376127362146161,-0.00692377066249325,0.00102859607467245,0.0
212253,64,KP.20241027.66888.25,autocal-etalon-all-morn,2024-10-27T18:35:18.259,1.0,0.00834584160181024,0.0294590226407886,-0.00844104980565607,0.00226682836111564,-0.00232322494623389,-0.000856250089147383,2460611.28001172,0.000735032332959909,-0.000990141650043638,0.00195033743039039,0.0
212254,65,KP.20241027.66888.25,autocal-etalon-all-morn,2024-10-27T18:35:18.259,1.0,0.0119953609660675,-0.0170567276105354,-0.00316852319312095,0.00264619279041688,-0.00163606581470596,0.0100843119272017,2460611.28001172,0.000844134889296413,0.00232391521720773,0.00224608027308953,0.0
